# 自己调试网络结构定义和训练过程
网络是lenet-5改
其实更像照着alexnet改的，如果是alexnet，norm全在pooling之后,lrn和BN确实不同，lrn是active和pooling之后


tf.bias_add、tf.matmul、tf.multiply都是可以重名的，如果不自定义名称，这些会形成一个数组，导致图形不连贯

todo:ema的IsVariableInitialized[0-11],也是重名数组，但是ema一般都是简单调用的两个操作，这一步被隐藏了，这个对整体结构图影响不大，应该是一个原地闭路，现在打印成开路了。可以手动拆，ungroup

# 反向传播
fer_forward定义在同一个笔记下，不import了。直接调用forward()

In [1]:
import tensorflow as tf
import numpy as np
NUM_CHANNELS = 1
# CONV1_KSIZE = 5
# CONV1_OUTPUT_CHANNEL = 64
# CONV2_KSIZE = 5
# CONV2_OUTPUT_CHANNEL = 64
# CONV3_KSIZE = 5
# CONV3_OUTPUT_CHANNEL = 128

FC_SIZE = 512
OUTPUT_NODE = 7
import fer_config as config
import fer_generator
import fer_forward
import time
BATCH_SIZE = 4#128
LEARNING_RATE_BASE = 0.0005
LEARNING_RATE_DECAY = 0.99
STEPS = 5
MOVING_AVERAGE_DECAY = 0.99
train_num_examples = 28709

In [2]:
def backward():
    global_step = tf.Variable(0, trainable = False)
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        train_num_examples / BATCH_SIZE,
        LEARNING_RATE_DECAY,
        staircase=True)
    x = tf.placeholder(tf.float32, [BATCH_SIZE, config.img_width, config.img_height, NUM_CHANNELS])
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE])
    
    y = fer_forward.forward(x,True)
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,labels=tf.argmax(y_,1))
    cem = tf.reduce_mean(ce,name='cross_entropy_reduce_mean')
    total_loss = tf.add(cem, tf.add_n(tf.get_collection('regularization_losses')), name='total_loss')
    
    train_step = tf.train.AdamOptimizer(learning_rate).minimize(total_loss, global_step = global_step)
    
    #打印graph的时候把ema屏蔽掉，不然有一些ema的初始化会把图弄的很乱
    ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step,name='ema')
    print('ema:',ema)
    ema_op = ema.apply(tf.trainable_variables())
    print('ema_op:',ema_op)
    
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)#BN需要依赖操作，但是为什么空？即使没有ema也是空。
    print(tf.global_variables())
    print('tf.GraphKeys.UPDATE_OPS：',tf.GraphKeys.UPDATE_OPS)
    print('update_ops:',update_ops)
    
    print('type of ema_op:',type(ema_op))
    print('type of update_ops:',type(update_ops))
    with tf.control_dependencies([train_step, ema_op]):
        train_op = tf.no_op(name='train')
    
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32),name='accuracy')#先转换类型再计算
    
    

    img_batch, label_batch = fer_generator.get_tfrecord(BATCH_SIZE, config.tfRecord_train)#位置没影响
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        tf.summary.FileWriter(logdir='my_net_graph', graph=sess.graph)#存网络


        print('will start!')
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
#         img_batch, label_batch = get_tfrecord(BATCH_SIZE, config.tfRecord_train, dbg = True)
        print(type(img_batch))
        for i in range(STEPS):#调试完，替换回STEPS
#             print(' in for loop')
            xs, ys = sess.run([img_batch, label_batch])
#             print(type(xs))
#             print(type(ys))
#             print('ys:',ys)
            reshaped_xs = np.reshape(xs, (BATCH_SIZE,config.img_width,config.img_height,NUM_CHANNELS))
            #数据出了点问题，train那个tfrecord0字节，可能有覆写
            _, loss_value, accuracy_value, step = sess.run([train_op, total_loss, accuracy, global_step],
                                                          feed_dict={x:reshaped_xs, y_:ys})
            print('loss:',loss_value)
            if (i + 1) % 2 == 0:
                print('%d'%(step))
                print('%g'%(loss_value))
                print('%g'%(accuracy_value))
                print("%s : After %d training step(s), loss,accuracy on training batch is %g , %g." % (time.strftime('%Y-%m-%d %H:%M:%S'), step, loss_value,accuracy_value))
        coord.request_stop()
        coord.join(threads)


In [3]:
backward()

#EMA有18变量，BN：3*2=6

ema: <tensorflow.python.training.moving_averages.ExponentialMovingAverage object at 0x7f7e2824ccf8>
ema_op: name: "ema"
op: "NoOp"
input: "^ema/AssignMovingAvg"
input: "^ema/AssignMovingAvg_1"
input: "^ema/AssignMovingAvg_2"
input: "^ema/AssignMovingAvg_3"
input: "^ema/AssignMovingAvg_4"
input: "^ema/AssignMovingAvg_5"
input: "^ema/AssignMovingAvg_6"
input: "^ema/AssignMovingAvg_7"
input: "^ema/AssignMovingAvg_8"
input: "^ema/AssignMovingAvg_9"
input: "^ema/AssignMovingAvg_10"
input: "^ema/AssignMovingAvg_11"
input: "^ema/AssignMovingAvg_12"
input: "^ema/AssignMovingAvg_13"
input: "^ema/AssignMovingAvg_14"
input: "^ema/AssignMovingAvg_15"
input: "^ema/AssignMovingAvg_16"
input: "^ema/AssignMovingAvg_17"

[<tf.Variable 'Variable:0' shape=() dtype=int32_ref>, <tf.Variable 'conv1_w:0' shape=(5, 5, 1, 64) dtype=float32_ref>, <tf.Variable 'conv1_b:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'bn1/gamma:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'bn1/beta:0' shape=(64,) dtype=float32

In [ ]:
# 读取tfRecord文件
def read_tfRecord(tfRecord_path):
    filename_queue = tf.train.string_input_producer([tfRecord_path])
    print('filename_queue:',filename_queue)
    reader = tf.TFRecordReader()
    _,serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label':tf.FixedLenFeature([7],tf.int64),
                                           'img_raw':tf.FixedLenFeature([],tf.string)
                                                 })
    img = tf.decode_raw(features['img_raw'],tf.uint8)
    img.set_shape([config.img_height * config.img_height])

    img = tf.cast(img,tf.float32)*(1./255)
    label = tf.cast(features['label'],tf.float32)
    return img,label


# 批量读取数据
def get_tfrecord(num, tfRecord_path,dbg = False):
    img,label=read_tfRecord(tfRecord_path)
    print('img readed:',img)
    print('label readed:',label)
    if dbg:#如果调试，不走shuffle
        return img, label
    img_batch,label_batch = tf.train.shuffle_batch([img,label],
                                                   batch_size =num,
                                                   num_threads=2,
                                                   capacity=10000,
                                                   min_after_dequeue=5000)
    print('return img_batch:',img_batch)
    print('return label_batch:',label_batch)
    return img_batch,label_batch

In [8]:
# help(tf.decode_raw)

Help on function decode_raw in module tensorflow.python.ops.gen_parsing_ops:

decode_raw(bytes, out_type, little_endian=True, name=None)
    Reinterpret the bytes of a string as a vector of numbers.
    
    Args:
      bytes: A `Tensor` of type `string`.
        All the elements must have the same length.
      out_type: A `tf.DType` from: `tf.half, tf.float32, tf.float64, tf.int32, tf.uint16, tf.uint8, tf.int16, tf.int8, tf.int64`.
      little_endian: An optional `bool`. Defaults to `True`.
        Whether the input `bytes` are in little-endian order.
        Ignored for `out_type` values that are stored in a single byte like
        `uint8`.
      name: A name for the operation (optional).
    
    Returns:
      A `Tensor` of type `out_type`.
      A Tensor with one more dimension than the input `bytes`.  The
      added dimension will have size equal to the length of the elements
      of `bytes` divided by the number of bytes to represent `out_type`.



## 专门调一下tfrecord的读取

In [ ]:

#     img_batch, label_batch = fer_generator.get_tfrecord(BATCH_SIZE, config.tfRecord_train)#位置没影响
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    print('will start!')
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    ####################################################
    filename_queue = tf.train.string_input_producer([config.tfRecord_train])
    print('filename_queue:',filename_queue)
    reader = tf.TFRecordReader()
    _,serialized_example = reader.read(filename_queue)
    print('serialized_example:',serialized_example)
    features = tf.parse_single_example(serialized_example,
                                       features={
                                           'label':tf.FixedLenFeature([7],tf.int64),
                                           'img_raw':tf.FixedLenFeature([],tf.string)
                                                 })
    print('features:',features)
    img = tf.decode_raw(features['img_raw'],tf.uint8)
    img.set_shape([config.img_height * config.img_height])

    img = tf.cast(img,tf.float32)*(1./255)
    label = tf.cast(features['label'],tf.float32)
#     img,label
    ###########################################
    #下面是shuffle
    img_batch,label_batch = tf.train.shuffle_batch([img,label],
                                                   batch_size = BATCH_SIZE,
                                                   num_threads=2,
                                                   capacity=10000,
                                                   min_after_dequeue=5000)

    #img_batch,label_batch
    
    #img_batch, label_batch = get_tfrecord(BATCH_SIZE, config.tfRecord_train, dbg = True)
    #############################################
    print(type(img))
    print(type(label))
    print(type(img_batch))
    print(type(label_batch))
    print(sess.run(serialized_example))
#     xs, ys = sess.run([img_batch, label_batch])
#     print(type(xs))
#     print(type(ys))
#     print('ys:',ys)
#     reshaped_xs = np.reshape(xs, (BATCH_SIZE,config.img_width,config.img_height,NUM_CHANNELS))
    #数据出了点问题，train那个tfrecord0字节，可能有覆写
#             _, loss_value, accuracy_value, step = sess.run([train_op, total_loss, accuracy, global_step],
#                                                           feed_dict={x:reshaped_xs, y_:ys})
#             print('loss:',loss_val)
#             if (i + 1) % 2 == 0:
#                 print("%s : After %d training step(s), loss,accuracy on training batch is %g , %g." % (
#                 time.strftime('%Y-%m-%d %H:%M:%S'), step, loss_value,accuracy_value))
#                 saver.save(sess, os.path.join(config.MODEL_SAVE_PATH, config.MODEL_NAME),global_step=global_step)
    coord.request_stop()
    coord.join(threads)

will start!
filename_queue: <tensorflow.python.ops.data_flow_ops.FIFOQueue object at 0x7fd99c4a7eb8>
serialized_example: Tensor("ReaderReadV2_1:1", shape=(), dtype=string)
features: {'img_raw': <tf.Tensor 'ParseSingleExample_1/Squeeze_img_raw:0' shape=() dtype=string>, 'label': <tf.Tensor 'ParseSingleExample_1/Squeeze_label:0' shape=(7,) dtype=int64>}
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>


In [ ]:
import tensorflow as tf
# help(tf.train.string_input_producer)
# help(tf.train.Features)

## FC中是否有必要用bias_add
不算batch的话，输出和bias已经是一对一了。
conv中要多一个维度，因为二维特征图的存在。
FC中两种都可以，CONV也允许,mnist lenet5的例子也没用bias_add，多虑了！

但是有一个情况记得是没通过，可能shape特殊或者没处理好。其实反而是用bias_add会报错，batch那一维丢了就不够条件了，可能是一些非batch的手动测试数据，出的问题。。。

In [30]:
import numpy as np
#模拟FC输出，batch=2
a = tf.constant(np.array([[1.,1.,2.],
                         [11.,11.,12.]
                         ]))
#模拟CONV输出，batch=2，三像素，2个像素点
#(2,2,2,3)
a2 = tf.constant(np.array([
    [[[1.,2.,3.],[4.,5.,6.]],
     [[7.,8.,9.],[10.,11.,12.]]],
    [[[11.,12.,13.],[14.,15.,16.]],
     [[17.,18.,19.],[20.,21.,22.]]]
                    ]))
#之前可能犯的错误，batch那一维没了，这样用bias_add是不允许的
a3 = tf.constant(np.array([1.,1.,2.]))
print(a)
print(a2)
b = tf.constant(np.array([0.1, 0.2, 0.3]))
c = tf.nn.bias_add(a,b)
d = a+b

c2 = tf.nn.bias_add(a2,b)
d2 = a2+b

# c3 = tf.nn.bias_add(a3,b)
d3 = a3+b

with tf.Session() as sess:
    print(sess.run(c))
    print(sess.run(d))
    print(sess.run(c2))
    print(sess.run(d2))
    print(sess.run(d3))
#     print(sess.run(c3))

Tensor("Const_49:0", shape=(2, 3), dtype=float64)
Tensor("Const_50:0", shape=(2, 2, 2, 3), dtype=float64)
[[  1.1   1.2   2.3]
 [ 11.1  11.2  12.3]]
[[  1.1   1.2   2.3]
 [ 11.1  11.2  12.3]]
[[[[  1.1   2.2   3.3]
   [  4.1   5.2   6.3]]

  [[  7.1   8.2   9.3]
   [ 10.1  11.2  12.3]]]


 [[[ 11.1  12.2  13.3]
   [ 14.1  15.2  16.3]]

  [[ 17.1  18.2  19.3]
   [ 20.1  21.2  22.3]]]]
[[[[  1.1   2.2   3.3]
   [  4.1   5.2   6.3]]

  [[  7.1   8.2   9.3]
   [ 10.1  11.2  12.3]]]


 [[[ 11.1  12.2  13.3]
   [ 14.1  15.2  16.3]]

  [[ 17.1  18.2  19.3]
   [ 20.1  21.2  22.3]]]]
[ 1.1  1.2  2.3]


In [47]:
#指定数值并指定shape，可以给每一个都初始化为指定数值/
a = tf.constant(0.0, dtype = tf.float32, shape=[CONV1_OUTPUT_CHANNEL])
print(a)

with tf.Session() as sess:
    print(sess.run(a))

Tensor("Const_54:0", shape=(64,), dtype=float32)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [34]:
#这个默认类型可以不指定
a = tf.zeros(shape=[OUTPUT_NODE], dtype=tf.float32)
a2 = tf.zeros(shape=[OUTPUT_NODE])
print(a)
print(a2)

Tensor("zeros_3:0", shape=(7,), dtype=float32)
Tensor("zeros_4:0", shape=(7,), dtype=float32)


In [9]:

# help(tf.add_n)
with tf.variable_scope('test2'):
    a = tf.constant(2.0, dtype = tf.float32, shape=[2],name='a')
    a2 = tf.constant(3.0, dtype = tf.float32, shape=[2],name='a2')
    tf.add_to_collection('test2', a)
    tf.add_to_collection('test2', a2)
b = tf.add_n(tf.get_collection('test2'))
print(a)

with tf.Session() as sess:
    print(sess.run(tf.get_collection('test2')))
    print(sess.run(a))
    print(sess.run(a2))
    print(sess.run(b))

Tensor("test2_2/a:0", shape=(2,), dtype=float32)
[array([ 2.,  2.], dtype=float32), array([ 3.,  3.], dtype=float32), array([ 2.,  2.], dtype=float32), array([ 3.,  3.], dtype=float32), array([ 2.,  2.], dtype=float32), array([ 3.,  3.], dtype=float32)]
[ 2.  2.]
[ 3.  3.]
[ 15.  15.]


# 为了找出BN的update操作为什么到了EMA做测试
实测，BN的average操作还在tf.GraphKeys.UPDATE_OPS中
另一个例子为什么不行了呢？

In [1]:

import tensorflow as tf
import fer_config as config
import fer_generator
import fer_forward
import time
NUM_CHANNELS = 1
OUTPUT_NODE = 7
BATCH_SIZE = 4#128对于虚拟机CPU来说，可能有点大。
LEARNING_RATE_BASE = 0.0005
LEARNING_RATE_DECAY = 0.99
STEPS = 1000
MOVING_AVERAGE_DECAY = 0.99
train_num_examples = 28709
def backward2():
    global_step = tf.Variable(0, trainable = False)
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        train_num_examples / BATCH_SIZE,
        LEARNING_RATE_DECAY,
        staircase=True)

    x = tf.placeholder(tf.float32, [BATCH_SIZE, config.img_width, config.img_height, NUM_CHANNELS])
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE])
    
    y = fer_forward.forward(x,True)
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,labels=tf.argmax(y_,1))
    cem = tf.reduce_mean(ce,name='cross_entropy_reduce_mean')
    total_loss = tf.add(cem, tf.add_n(tf.get_collection('regularization_losses')), name='total_loss')
    
    train_step = tf.train.AdamOptimizer(0.01).minimize(total_loss)#很多变量Adam有两次操作，因为正则化

    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    print(tf.global_variables())
    print('update_ops:',update_ops)
    
    ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step,name='ema')
    print('\nema:',ema)
    ema_op = ema.apply(tf.trainable_variables())
    print('\nema_op:',ema_op)
    
    print('type:',type(train_step), type(update_ops))
    
    if 1:
#     with tf.control_dependencies([train_step, update_ops]):#不行，tensor和list的组合如何写？分开依赖！或者别用get_collection！保持ops类型
        with tf.control_dependencies([train_step, tf.GraphKeys.UPDATE_OPS]):
            train_op = tf.no_op(name='train')
    else:
        with tf.control_dependencies( update_ops):
            train_op = tf.no_op(name='update_ops')
        with tf.control_dependencies([train_step, train_op]):
            train_op = tf.no_op(name='train')
    
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32),name='accuracy')#先转换类型再计算
    
backward2() 

[<tf.Variable 'Variable:0' shape=() dtype=int32_ref>, <tf.Variable 'conv1_w:0' shape=(5, 5, 1, 64) dtype=float32_ref>, <tf.Variable 'conv1_b:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'bn1/gamma:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'bn1/beta:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'bn1/moving_mean:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'bn1/moving_variance:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'conv2_w:0' shape=(5, 5, 64, 64) dtype=float32_ref>, <tf.Variable 'conv2_b:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'bn2/gamma:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'bn2/beta:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'bn2/moving_mean:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'bn2/moving_variance:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'conv3_w:0' shape=(5, 5, 64, 128) dtype=float32_ref>, <tf.Variable 'conv3_b:0' shape=(128,) dtype=float32_ref>, <tf.Variable 'bn3/gamma:0' shape=(128,) dtype=float32_ref>, <tf.Variable '

KeyError: "The name 'update_ops' refers to an Operation not in the graph."

In [8]:
# help(tf.nn.l2_loss)#output = sum(t ** 2) / 2

In [28]:
# help(tf.nn.conv2d)#先kernel，再inout
# help(tf.nn.lrn)
# help(tf.train.Saver)

# 练习：truncated数据初始化，观察数据规律
tf_demo/api_demo/truncated_normal.ipynb

In [1]:
import fer_config as config
config.MODEL_SAVE_PATH


'E:/MachineLearning/model_saved/fer2013'

In [3]:
import cv2
cv2.__version__

'3.4.2'

In [4]:
import PIL
PIL.__version__

'5.4.1'

In [6]:
import tensorflow as tf
help(tf.summary.histogram)

Help on function histogram in module tensorflow.python.summary.summary:

histogram(name, values, collections=None, family=None)
    Outputs a `Summary` protocol buffer with a histogram.
    
    Adding a histogram summary makes it possible to visualize your data's
    distribution in TensorBoard. You can see a detailed explanation of the
    TensorBoard histogram dashboard
    [here](https://www.tensorflow.org/get_started/tensorboard_histograms).
    
    The generated
    [`Summary`](https://www.tensorflow.org/code/tensorflow/core/framework/summary.proto)
    has one summary value containing a histogram for `values`.
    
    This op reports an `InvalidArgument` error if any value is not finite.
    
    Args:
      name: A name for the generated node. Will also serve as a series name in
        TensorBoard.
      values: A real numeric `Tensor`. Any shape. Values to use to
        build the histogram.
      collections: Optional list of graph collections keys. The new summary op is
 